<a href="https://colab.research.google.com/github/snjauregui/notebooks/blob/main/TP2_Properati.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## machete
- simple (ver cual explica mejor el precio)
- multiple
- regularizada


ver si ajustando una segmentación tenemos menos missing

Variable a predecir
- price_usd_per_m2 (volar los nulos)


Variables predictoras
- surface_total_in_m2
- property_type
- place_name (ver de agrupar para bajar la cardinalidad, ver si se pueden sacar los valores raros)
- cochera
- amenities



agregar cuadros de texto con la explicacion
dividir mejor limpieza y analisis
- exploracion
- limpieza
- analisis

Secciones TP2
- ET (Limpieza, transformaciones, imputaciones, drops, documentar las decisiones)
- Analisis
- Split
- Entrenamiento
- Test


Revisar cua




# Setup

## Library Imports

In [69]:
#from google.colab import drive
#drive.mount('/content/drive')
#!ls

In [70]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
 
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [71]:
!pip install --upgrade geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
%matplotlib inline
import seaborn as sns
from IPython.display import Image
import re
import plotly.express as px
import geopandas as gpd


print("Setup Complete")



Setup Complete


## Load data

In [73]:
#Acceso a google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
#Cargar el dataset
properatti_data_filepath = '/content/drive/MyDrive/Colab Notebooks/Data Science - Digital House/TP1/properatti.csv'
properatti_data = pd.read_csv(properatti_data_filepath)

#properatti_data = pd.read_csv('/content/properatti.csv')

pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 75


properatti_data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mataderos_lavadero_patio_inm...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PA...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIKufMJMwHtquzhk=/trim/198x0...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_departamentos_la-plata_balcon_l...,"Venta de departamento en décimo piso al frente, a estrenar. Living come...",VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv45KAKirGL3JM8=/trim/198x0...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_departamentos_mataderos_lavader...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON ...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_Ejwfu6QpW4cH0=/trim/198x0...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_liniers_patio_g-goffredo,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso....",PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SPyleyeQlIypI8w=/trim/198x0...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_departamentos_centro_cristina-p...,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIF...",DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA MITRE,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f5l8TrxdY-0kvs=/trim/198x0...


# Limpieza

## Borro las columnas que no usamos

In [75]:
properatti_data = properatti_data[[
 'property_type',
 'place_name',
 'country_name',
 'lat',
 'lon',
 'price',
 'currency',
 'state_name',
 'price_aprox_usd',
 'surface_total_in_m2',
 'surface_covered_in_m2',
 'price_usd_per_m2',
 'price_per_m2',
 'description']]

properatti_data.head()

,property_type,place_name,country_name,lat,lon,price,currency,state_name,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,description
0,PH,Mataderos,Argentina,-34.661824,-58.508839,62000.0,USD,Capital Federal,62000.0,55.0,40.0,1127.272727,1550.000000,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PA..."
1,apartment,La Plata,Argentina,-34.903883,-57.964330,150000.0,USD,Bs.As. G.B.A. Zona Sur,150000.0,NaN,NaN,NaN,NaN,"Venta de departamento en décimo piso al frente, a estrenar. Living come..."
2,apartment,Mataderos,Argentina,-34.652262,-58.522982,72000.0,USD,Capital Federal,72000.0,55.0,55.0,1309.090909,1309.090909,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON ...
3,PH,Liniers,Argentina,-34.647797,-58.516424,95000.0,USD,Capital Federal,95000.0,NaN,NaN,NaN,NaN,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso...."
4,apartment,Centro,Argentina,-38.002626,-57.549447,64000.0,USD,Buenos Aires Costa Atlántica,64000.0,35.0,35.0,1828.571429,1828.571429,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIF..."


## Borro las registros que no tienen dato de precio

Cuando no tienen currency tampoco tienen los otros campos de price, es lo mismo cual elegís para limpiar

In [76]:
properatti_data = properatti_data[properatti_data['currency'].notnull()]
properatti_data.head(5)

,property_type,place_name,country_name,lat,lon,price,currency,state_name,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,description
0,PH,Mataderos,Argentina,-34.661824,-58.508839,62000.0,USD,Capital Federal,62000.0,55.0,40.0,1127.272727,1550.000000,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PA..."
1,apartment,La Plata,Argentina,-34.903883,-57.964330,150000.0,USD,Bs.As. G.B.A. Zona Sur,150000.0,NaN,NaN,NaN,NaN,"Venta de departamento en décimo piso al frente, a estrenar. Living come..."
2,apartment,Mataderos,Argentina,-34.652262,-58.522982,72000.0,USD,Capital Federal,72000.0,55.0,55.0,1309.090909,1309.090909,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON ...
3,PH,Liniers,Argentina,-34.647797,-58.516424,95000.0,USD,Capital Federal,95000.0,NaN,NaN,NaN,NaN,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso...."
4,apartment,Centro,Argentina,-38.002626,-57.549447,64000.0,USD,Buenos Aires Costa Atlántica,64000.0,35.0,35.0,1828.571429,1828.571429,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIF..."


In [77]:
properatti_data.price_aprox_usd.count()

100809

## Borramos propiedades con la descripcion duplicada

In [78]:
properatti_data = properatti_data.drop_duplicates(subset=['description'], keep="first")

In [79]:
properatti_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88392 entries, 0 to 121219
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   property_type          88392 non-null  object 
 1   place_name             88369 non-null  object 
 2   country_name           88392 non-null  object 
 3   lat                    52482 non-null  float64
 4   lon                    52482 non-null  float64
 5   price                  88392 non-null  float64
 6   currency               88392 non-null  object 
 7   state_name             88392 non-null  object 
 8   price_aprox_usd        88392 non-null  float64
 9   surface_total_in_m2    59351 non-null  float64
 10  surface_covered_in_m2  76468 non-null  float64
 11  price_usd_per_m2       59102 non-null  float64
 12  price_per_m2           76467 non-null  float64
 13  description            88391 non-null  object 
dtypes: float64(8), object(6)
memory usage: 10.1+ MB


## Borro las publicaciones que no están en ARS o USD

In [80]:
properatti_data = properatti_data[properatti_data['currency'].isin(['ARS','USD'])]
properatti_data.head()

,property_type,place_name,country_name,lat,lon,price,currency,state_name,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,description
0,PH,Mataderos,Argentina,-34.661824,-58.508839,62000.0,USD,Capital Federal,62000.0,55.0,40.0,1127.272727,1550.000000,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PA..."
1,apartment,La Plata,Argentina,-34.903883,-57.964330,150000.0,USD,Bs.As. G.B.A. Zona Sur,150000.0,NaN,NaN,NaN,NaN,"Venta de departamento en décimo piso al frente, a estrenar. Living come..."
2,apartment,Mataderos,Argentina,-34.652262,-58.522982,72000.0,USD,Capital Federal,72000.0,55.0,55.0,1309.090909,1309.090909,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON ...
3,PH,Liniers,Argentina,-34.647797,-58.516424,95000.0,USD,Capital Federal,95000.0,NaN,NaN,NaN,NaN,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso...."
4,apartment,Centro,Argentina,-38.002626,-57.549447,64000.0,USD,Buenos Aires Costa Atlántica,64000.0,35.0,35.0,1828.571429,1828.571429,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIF..."


## Columnas nuevas

### Amenities

In [81]:
#No amenities:
#Escalera
#Baulera
#Iluminacion
#Jardin


###amenities a considerar:
#Ascensor
#Pileta
#Cochera+garage+estacionamiento + guarda coche + guardacoche
#Parrilla 
#Laundry
#SUM
#Balcon


In [82]:
#AA-AMENITIES
#Se evaluan los posibles Amenities, buscando en columna "description" y se agrega en un nuevo DataFrame una columna por Amenitie o 
#por alguna caracteristica de la propiedad; la busqueda se hace sin considerar discriminacion por Minuscula/Mayuscula con case = False


#No Amenities

#Escaleras
esc = properatti_data.description.str.contains("escalera", case=False)
properatti_data["Escalera"] = esc
properatti_data.value_counts('Escalera')

#Validacion
#Con_Escalera = properatti_data['Escalera'] == True
#properatti_data.value_counts(Con_Escalera)
#pd.options.display.max_colwidth = 250
#properatti_data.description[Con_Escalera]


#Baulera
baul = properatti_data.description.str.contains(" Baulera", case=False)
properatti_data["Baulera"] = baul
properatti_data.value_counts('Baulera')

#iluminacion
ilum= ['iluminacion', 'iluminación']
iluminacion = properatti_data.description.str.contains('|'.join(ilum), case=False)
properatti_data["Iluminacion"] = iluminacion
properatti_data.value_counts('Iluminacion')

#Jardin
jard = properatti_data.description.str.contains(" Jardin ", case=False)
properatti_data["Jardin"] = jard
properatti_data.value_counts('Jardin')



#Amenities

#Elevador
elev= ['elevador', 'ascensor'] #Posibles valores (sinonimos) para buscar
ascensor = properatti_data.description.str.contains('|'.join(elev), case=False)
properatti_data["Ascensor"] = ascensor
properatti_data.value_counts('Ascensor')

#Pile
pile= ['pileta', 'piscina']
pileta = properatti_data.description.str.contains('|'.join(pile), case=False)
properatti_data["Pileta"] = pileta
properatti_data.value_counts('Pileta')

#Cochera
coche= ['Cochera', 'garage', 'estacionamiento', 'guarda coche', 'guardacoche']
cochera = properatti_data.description.str.contains('|'.join(coche), case=False)
properatti_data["Cochera"] = cochera
properatti_data.value_counts('Cochera')

#Parrilla
parri= ['Parrilla']
Parrilla = properatti_data.description.str.contains('|'.join(parri), case=False)
properatti_data["Parrilla"] = Parrilla
properatti_data.value_counts('Parrilla')


#Laundry
laun = properatti_data.description.str.contains("laundry", case=False)
properatti_data["Laundry"] = laun
properatti_data.value_counts('Laundry')

#SUM
sum_ = properatti_data.description.str.contains(" sum ", case=False)
properatti_data["SUM"] = sum_
properatti_data.value_counts('SUM')

#Balcon
balc= [' Balcon', ' Balcón']
Balcon = properatti_data.description.str.contains('|'.join(balc), case=False)
#pileta.value_counts()
properatti_data["Balcon"] = Balcon
properatti_data.value_counts('Balcon')

#Amenities: Solo considerar columnas Ascensor, Pileta, Cochera, Parrilla, Laundry, SUM, Balcon
Amenitie = ((properatti_data['Ascensor'] == True) | (properatti_data['Pileta'] == True) | 
(properatti_data['Cochera'] == True) | (properatti_data['Parrilla'] == True) | 
(properatti_data['Laundry'] == True) | (properatti_data['SUM'] == True) | (properatti_data['Balcon'] == True))

properatti_data["Amenities"] = Amenitie
properatti_data.value_counts('Amenities')


Amenities
True     64160
False    24229
dtype: int64

## Check nulos post limpieza

In [83]:
properatti_data_caba = properatti_data[properatti_data["state_name"] == 'Capital Federal']


percent_missing = properatti_data_caba.isnull().sum() * 100 / len(properatti_data_caba)
missing_value_df = pd.DataFrame({'column_name': properatti_data_caba.columns,
                                 'percent_missing': percent_missing})
missing_value_df


,column_name,percent_missing
property_type,property_type,0.000000
place_name,place_name,0.000000
country_name,country_name,0.000000
lat,lat,20.876491
lon,lon,20.876491
price,price,0.000000
currency,currency,0.000000
state_name,state_name,0.000000
price_aprox_usd,price_aprox_usd,0.000000
surface_total_in_m2,surface_total_in_m2,21.784344


In [84]:
properatti_data_caba.place_name.unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo Soho', 'Palermo',
       'Flores', 'Boedo', 'Las Cañitas', 'Puerto Madero', 'Balvanera',
       'Caballito', 'Nuñez', 'San Telmo', 'Almagro', 'Capital Federal',
       'Colegiales', 'Floresta', 'Barrio Norte', 'Barracas', 'Recoleta',
       'Congreso', 'Villa Crespo', 'Chacarita', 'Constitución',
       'Villa Urquiza', 'Palermo Hollywood', 'Saavedra', 'Monserrat',
       'Pompeya', 'Parque Chas', 'Paternal', 'Agronomía',
       'Villa Pueyrredón', 'Coghlan', 'Parque Centenario', 'Villa Luro',
       'Villa Devoto', 'Boca', 'Parque Avellaneda', 'San Cristobal',
       'Abasto', 'Versalles', 'Villa del Parque', 'Monte Castro',
       'Retiro', 'Parque Patricios', 'San Nicolás', 'Villa Santa Rita',
       'Palermo Chico', 'Centro / Microcentro', 'Tribunales', 'Once',
       'Parque Chacabuco', 'Velez Sarsfield', 'Catalinas',
       'Villa General Mitre', 'Palermo Viejo', 'Villa Lugano',
       'Villa Ortuzar', 'Villa Soldati', 'Villa Re

## Borramos los barrios con valor Capital Federal

In [85]:
properatti_data_caba = properatti_data_caba[properatti_data_caba["place_name"] != 'Capital Federal']

properatti_data_caba.place_name.unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo Soho', 'Palermo',
       'Flores', 'Boedo', 'Las Cañitas', 'Puerto Madero', 'Balvanera',
       'Caballito', 'Nuñez', 'San Telmo', 'Almagro', 'Colegiales',
       'Floresta', 'Barrio Norte', 'Barracas', 'Recoleta', 'Congreso',
       'Villa Crespo', 'Chacarita', 'Constitución', 'Villa Urquiza',
       'Palermo Hollywood', 'Saavedra', 'Monserrat', 'Pompeya',
       'Parque Chas', 'Paternal', 'Agronomía', 'Villa Pueyrredón',
       'Coghlan', 'Parque Centenario', 'Villa Luro', 'Villa Devoto',
       'Boca', 'Parque Avellaneda', 'San Cristobal', 'Abasto',
       'Versalles', 'Villa del Parque', 'Monte Castro', 'Retiro',
       'Parque Patricios', 'San Nicolás', 'Villa Santa Rita',
       'Palermo Chico', 'Centro / Microcentro', 'Tribunales', 'Once',
       'Parque Chacabuco', 'Velez Sarsfield', 'Catalinas',
       'Villa General Mitre', 'Palermo Viejo', 'Villa Lugano',
       'Villa Ortuzar', 'Villa Soldati', 'Villa Real', 'Villa Riachue

## Normalizamos los barrios con caracteres extraños

In [87]:
##AA Normalizacion
#Normalizacion de Barrios por error de Acentos y Ñ
properatti_data_caba.place_name = properatti_data_caba.place_name.replace(['Las CaÃ±itas', 'NuÃ±ez', 'ConstituciÃ³n', 'AgronomÃ\xada', 'Villa PueyrredÃ³n', 
                                                                 'San NicolÃ¡s'],
                      ['Las Canitas', 'Nunez', 'Constitucion', 'Agronomia', 'Villa Pueyrredon','San Nicolas'])

properatti_data_caba.place_name.unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo Soho', 'Palermo',
       'Flores', 'Boedo', 'Las Cañitas', 'Puerto Madero', 'Balvanera',
       'Caballito', 'Nuñez', 'San Telmo', 'Almagro', 'Colegiales',
       'Floresta', 'Barrio Norte', 'Barracas', 'Recoleta', 'Congreso',
       'Villa Crespo', 'Chacarita', 'Constitución', 'Villa Urquiza',
       'Palermo Hollywood', 'Saavedra', 'Monserrat', 'Pompeya',
       'Parque Chas', 'Paternal', 'Agronomía', 'Villa Pueyrredón',
       'Coghlan', 'Parque Centenario', 'Villa Luro', 'Villa Devoto',
       'Boca', 'Parque Avellaneda', 'San Cristobal', 'Abasto',
       'Versalles', 'Villa del Parque', 'Monte Castro', 'Retiro',
       'Parque Patricios', 'San Nicolás', 'Villa Santa Rita',
       'Palermo Chico', 'Centro / Microcentro', 'Tribunales', 'Once',
       'Parque Chacabuco', 'Velez Sarsfield', 'Catalinas',
       'Villa General Mitre', 'Palermo Viejo', 'Villa Lugano',
       'Villa Ortuzar', 'Villa Soldati', 'Villa Real', 'Villa Riachue

## Mapeamos valores que corresponden al mismo barrio

In [88]:
#Normalización de Nombres de Barrios

properatti_data_caba.place_name = properatti_data_caba.place_name.replace(['Palermo Soho', 'Palermo Hollywood', 'Palermo Chico', 'Palermo Viejo', 'Las Cañitas', 'Catalinas', 'Boca'],
                      ['Palermo', 'Palermo', 'Palermo', 'Palermo', 'Palermo', 'Retiro', 'La Boca'])

    
properatti_data_caba.place_name.unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo', 'Flores', 'Boedo',
       'Puerto Madero', 'Balvanera', 'Caballito', 'Nuñez', 'San Telmo',
       'Almagro', 'Colegiales', 'Floresta', 'Barrio Norte', 'Barracas',
       'Recoleta', 'Congreso', 'Villa Crespo', 'Chacarita',
       'Constitución', 'Villa Urquiza', 'Saavedra', 'Monserrat',
       'Pompeya', 'Parque Chas', 'Paternal', 'Agronomía',
       'Villa Pueyrredón', 'Coghlan', 'Parque Centenario', 'Villa Luro',
       'Villa Devoto', 'La Boca', 'Parque Avellaneda', 'San Cristobal',
       'Abasto', 'Versalles', 'Villa del Parque', 'Monte Castro',
       'Retiro', 'Parque Patricios', 'San Nicolás', 'Villa Santa Rita',
       'Centro / Microcentro', 'Tribunales', 'Once', 'Parque Chacabuco',
       'Velez Sarsfield', 'Villa General Mitre', 'Villa Lugano',
       'Villa Ortuzar', 'Villa Soldati', 'Villa Real', 'Villa Riachuelo'],
      dtype=object)

## Chequeamos la cantidad de registros por barrio

In [89]:

List_place = properatti_data_caba.place_name
counts = List_place.value_counts()
percent = List_place.value_counts(normalize=True)

#DataFrame con Totales y Porcentajes de 2 decimales por cada Place
Place_Porcentaje= pd.DataFrame({'Totales': counts, "per100": np.round((percent*100),2)})

Place_Porcentaje.head(3)

#Se define una lista con Place Unicos
Place_distinct = List_place.unique()
Place_Unique = Place_distinct

#Se define una lista con Porcentaje de cada Place
Porcentaje_Place = Place_Porcentaje.per100

Porcentaje_Place.head(3)


#Se arma un Dataframe con Place que tengan un Porcentaje 
D_PN = Place_Porcentaje['per100'] >=0
D_PN.head(2)
Df_SP_Place = pd.DataFrame({'Place': Place_Porcentaje.loc[D_PN, :].index, 'Porcentaje': Place_Porcentaje.loc[D_PN, :].per100})
Df_SP_Place


,Place,Porcentaje
Palermo,Palermo,13.33
Belgrano,Belgrano,8.97
Caballito,Caballito,6.89
Recoleta,Recoleta,5.47
Villa Urquiza,Villa Urquiza,4.39
Flores,Flores,4.21
Villa Crespo,Villa Crespo,4.02
Barrio Norte,Barrio Norte,3.93
Almagro,Almagro,3.63
Boedo,Boedo,3.20


In [90]:
import plotly.express as px

plotly_df = px.data.gapminder()
fig = px.bar(plotly_df, x=Df_SP_Place.Place, y=Df_SP_Place.Porcentaje,
             labels={'Place_name': 'Place', 
                     'Porcentaje': '% Porcentajes'}, height=400,  
             title="% de Propiedades a la Venta por Place en CABA")


fig.show()

## Mapeamos los barrios a sus comunas para obtener una dimension de menor cardinalidad

In [127]:
#Asignar en nueva columna la Comuna
Condiciones = [
    ((properatti_data_caba.place_name =='Retiro') | (properatti_data_caba.place_name =='San Nicolas') | (properatti_data_caba.place_name =='San Nicolás') | (properatti_data_caba.place_name =='Puerto Madero') | (properatti_data_caba.place_name =='San Telmo') | (properatti_data_caba.place_name =='Montserrat') | (properatti_data_caba.place_name =='Monserrat') | (properatti_data_caba.place_name =='Constitucion')| (properatti_data_caba.place_name =='Constitución')| (properatti_data_caba.place_name =='Centro / Microcentro')| (properatti_data_caba.place_name =='Tribunales')),
    (properatti_data_caba.place_name =='Recoleta') | (properatti_data_caba.place_name =='Barrio Norte'),
    ((properatti_data_caba.place_name =='Balvanera') | (properatti_data_caba.place_name =='San Cristobal') | (properatti_data_caba.place_name =='Congreso')| (properatti_data_caba.place_name =='Abasto')| (properatti_data_caba.place_name =='Once')),
    ((properatti_data_caba.place_name =='La Boca') | (properatti_data_caba.place_name =='Barracas') | (properatti_data_caba.place_name =='Parque Patricios') | (properatti_data_caba.place_name =='Nueva Pompeya')| (properatti_data_caba.place_name =='Pompeya')),
    ((properatti_data_caba.place_name =='Almagro') | (properatti_data_caba.place_name =='Boedo')),
    ((properatti_data_caba.place_name =='Caballito') | (properatti_data_caba.place_name =='Parque Centenario')),
    ((properatti_data_caba.place_name =='Flores') | (properatti_data_caba.place_name =='Parque Chacabuco')),
    ((properatti_data_caba.place_name =='Villa Soldati') | (properatti_data_caba.place_name =='Villa Riachuelo') | (properatti_data_caba.place_name =='Villa Lugano')),
    ((properatti_data_caba.place_name =='Liniers') | (properatti_data_caba.place_name =='Mataderos') | (properatti_data_caba.place_name =='Parque Avellaneda')),
    ((properatti_data_caba.place_name =='Villa Real') | (properatti_data_caba.place_name =='Monte Castro') | (properatti_data_caba.place_name =='Versalles') | (properatti_data_caba.place_name =='Floresta') | (properatti_data_caba.place_name =='Velez Sarsfield') | (properatti_data_caba.place_name =='Villa Luro')),
    ((properatti_data_caba.place_name =='Villa General Mitre') | (properatti_data_caba.place_name =='Villa Devoto') | (properatti_data_caba.place_name =='Villa del Parque') | (properatti_data_caba.place_name =='Villa Santa Rita')),
    ((properatti_data_caba.place_name =='Coghlan') | (properatti_data_caba.place_name =='Saavedra') | (properatti_data_caba.place_name =='Villa Urquiza') | (properatti_data_caba.place_name =='Villa Pueyrredon')| (properatti_data_caba.place_name =='Villa Pueyrredón')),
    ((properatti_data_caba.place_name =='Nuñez') | (properatti_data_caba.place_name =='Nunez') | (properatti_data_caba.place_name =='Belgrano') | (properatti_data_caba.place_name =='Colegiales')),
    (properatti_data_caba.place_name =='Palermo'),
    ((properatti_data_caba.place_name =='Chacarita') | (properatti_data_caba.place_name =='Villa Crespo') | (properatti_data_caba.place_name =='La Paternal') | (properatti_data_caba.place_name =='Paternal')| (properatti_data_caba.place_name =='Villa Ortuzar') | (properatti_data_caba.place_name =='Agronomia') | (properatti_data_caba.place_name =='Parque Chas')| (properatti_data_caba.place_name =='Agronomía'))
]

Opciones = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

properatti_data_caba['Comuna']=np.select(Condiciones,Opciones)

properatti_data_caba.head(5)

,property_type,place_name,country_name,lat,lon,price,currency,state_name,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,description,Escalera,Baulera,Iluminacion,Jardin,Ascensor,Pileta,Cochera,Parrilla,Laundry,SUM,Balcon,Amenities,Comuna
0,PH,Mataderos,Argentina,-34.661824,-58.508839,62000.0,USD,Capital Federal,62000.0,55.0,40.0,1127.272727,1550.000000,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PA...",False,False,False,False,False,False,False,False,False,False,False,False,9
2,apartment,Mataderos,Argentina,-34.652262,-58.522982,72000.0,USD,Capital Federal,72000.0,55.0,55.0,1309.090909,1309.090909,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON ...,False,False,False,False,False,False,False,False,False,False,False,False,9
3,PH,Liniers,Argentina,-34.647797,-58.516424,95000.0,USD,Capital Federal,95000.0,NaN,NaN,NaN,NaN,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso....",False,False,False,False,False,False,False,False,False,False,False,False,9
7,apartment,Belgrano,Argentina,-34.559873,-58.443362,138000.0,USD,Capital Federal,138000.0,45.0,40.0,3066.666667,3450.000000,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER LUMINOSO ...PISO ALTO CO...,False,False,False,False,False,True,False,False,True,False,False,True,13
8,apartment,Belgrano,Argentina,-34.559873,-58.443362,195000.0,USD,Capital Federal,195000.0,65.0,60.0,3000.000000,3250.000000,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER LUMINOSO ...PISO ALTO CO...,False,False,False,False,False,True,False,False,True,False,False,True,13


In [128]:
List_comuna = properatti_data_caba.Comuna
counts_c = List_comuna.value_counts()
percent = List_comuna.value_counts(normalize=True)

#DataFrame con Totales y Porcentajes de 2 decimales por cada Comuna
Comuna_Porcentaje= pd.DataFrame({'Totales': counts_c, "per100": np.round((percent*100),2)})

Comuna_Porcentaje.head(3)

#Se define una lista con Comuna Unicos
Comuna_distinct = List_comuna.unique()
Comuna_Unique = Comuna_distinct

#Se define una lista con Porcentaje de cada Comuna
Porcentaje_Comuna = Comuna_Porcentaje.per100

Porcentaje_Comuna.head(3)


#Se arma un Dataframe con Comunas que tengan un Porcentaje 
D_Com = Comuna_Porcentaje['per100'] >=0
D_Com.head(2)
Df_SP_Comuna = pd.DataFrame({'Comuna': Comuna_Porcentaje.loc[D_Com, :].index, 'Porcentaje': Comuna_Porcentaje.loc[D_Com, :].per100})
Res_Comuna = Df_SP_Comuna.sort_values('Porcentaje',ascending=False)
Res_Comuna

,Comuna,Porcentaje
14,14,13.33
13,13,12.90
1,1,10.20
2,2,9.39
12,12,7.82
6,6,7.09
3,3,6.87
5,5,6.83
15,15,5.78
7,7,4.75


In [129]:
plotly_df_comunas = px.data.gapminder()
fig = px.bar(plotly_df_comunas, x=Res_Comuna.Comuna, y=Res_Comuna.Porcentaje,
             labels={'Comuna': 'Comuna', 
                     'Porcentaje': '% Porcentajes'}, height=400,  
             title="% de Propiedades a la Venta por Comuna en CABA")


fig.show()